In [12]:
import cv2
import numpy as np
import os

In [13]:
# ⚠️ Set these to your local image paths
reference_path = "C:\\Users\\Darshan\\Downloads\\Computer Vision\\CrossWalkPedestrian.jpg"
query_path = "C:\\Users\\Darshan\\Downloads\\Computer Vision\\CrossWalkPedestrianSingle.jpg"

In [14]:
# Load reference and query images
reference_img = cv2.imread(reference_path, cv2.IMREAD_GRAYSCALE)
query_img = cv2.imread(query_path, cv2.IMREAD_GRAYSCALE)

In [15]:
# Validate images
if reference_img is None:
    print(f"❌ Could not load reference image from '{reference_path}'")
    exit()

In [16]:
if query_img is None:
    print(f"❌ Could not load query image from '{query_path}'")
    exit()

In [17]:
# Initialize SIFT detector
sift = cv2.SIFT_create()

In [18]:
# Detect keypoints and compute descriptors
kp1, des1 = sift.detectAndCompute(reference_img, None)
kp2, des2 = sift.detectAndCompute(query_img, None)

In [19]:
# Create BFMatcher and find matches
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)

In [20]:
# Apply Lowe's ratio test
good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]

In [21]:
# Proceed only if enough good matches are found
if len(good_matches) >= 10:
    # Extract matched keypoints
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Find homography using RANSAC
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    if mask is not None:
        inliers = mask.ravel().tolist()
        num_inliers = np.sum(inliers)

        # Draw inlier matches
        inlier_matches = [good_matches[i] for i in range(len(good_matches)) if inliers[i]]
        match_img = cv2.drawMatches(reference_img, kp1, query_img, kp2, inlier_matches, None, flags=2)

        # Show result
        cv2.namedWindow("Matched Keypoints", cv2.WINDOW_NORMAL)
        cv2.imshow("Matched Keypoints", match_img)
        print(f"Number of inliers: {num_inliers}")

        if num_inliers > 15:
            print("✅ Person detected!")
        else:
            print("❌ No person detected.")

        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("❌ Homography could not be computed.")
else:
    print("❌ Not enough good matches found to proceed.")

Number of inliers: 406
✅ Person detected!
